Loading the data

id: household

Choices:  
- heinz41  
- heinz32  
- heinz28  
- hunts32  

variables  
- disp (1/0)  
- feat (1/0)  
- price (Log?)  

In [2]:
import numpy as np
import pandas as pd
#pd.options.mode.chained_assignment = None
df = pd.read_pickle("data.pkl") 
df.head()

,id,disp.heinz41,disp.heinz32,disp.heinz28,disp.hunts32,feat.heinz41,feat.heinz32,feat.heinz28,feat.hunts32,price.heinz41,price.heinz32,price.heinz28,price.hunts32,choice,choiceindex
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28,2
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz28,2
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz28,2
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28,2
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz28,2


Generating data to benchmark

In [201]:
%%time
import pickle
### Configuration ###########

#number of datasets
n = 10

# True coefficients
c = [1.8, #alpha heinz41
     1.2, #alpha heinz32 
     2.0, #alpha heinz28
     1.3, #mu    display
     1.1, #mu    feat
     -1.8, #mu    price
     0.25, #sigma dispay 
     0.5, #sigma feat 
     1.2 #sigma price 
    ]

seed = 1234

#############################

np.random.seed(seed)
#brand choices
choices = ["heinz41", "heinz32", "heinz28", "hunts32"]

#generating the beta per household

def drawbeta(household):
    return (household + c[3:6]) * c[6:9]
            #std        #mu       #sigma

beta = np.apply_along_axis(drawbeta, 1, np.random.randn(300, 3, n))

#defining the choice making process
def makechoice(row, index):
    utilities = np.zeros(4)
    #calculate utility for each brand
    for k in range(4):
        #get the x observation
        x = [row[1+k], row[5+k], row[9+k]]

        #start with the base prefrence
        utility = 0; #start with alpha
        if k < 3: utility = c[k]
        
        #calulate the indiviual utility
        utility += np.dot(x, beta[row.id-1, :, index]) 
        
        #add the error of extreme value???
        
        #save the utility
        utilities[k] = utility

    #pick and set the highest utility brand
    row.choiceindex = np.argmax(utilities)
    row.choice = choices[row.choiceindex]
    return row


#generating the dataset with the dgp
data = [df.copy().apply(makechoice, axis=1, index=r) for r in range(n)]

#organsing the dataset
dataset = {
    "n":n,
    "c":c,
    "seed": seed,
    "beta": beta,
    "data": data  
}

#saving the datased
np.save("test", dataset)

Wall time: 7.94 s


In [200]:
dataset['data'][0].head()

,id,disp.heinz41,disp.heinz32,disp.heinz28,disp.hunts32,feat.heinz41,feat.heinz32,feat.heinz28,feat.hunts32,price.heinz41,price.heinz32,price.heinz28,price.hunts32,choice,choiceindex
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz32,1
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz32,1
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz32,1
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz32,1
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz32,1


In [8]:
ca = np.load('test.npy')

In [17]:
ca

array({'n': 10, 'c': [1.8, 1.2, 2.0, 1.3, 1.1, -1.8, 0.25, 0.5, 1.2], 'seed': 1234, 'beta': array([[[ 0.44285879,  0.02725608,  0.68317674, ...,  0.16586912,
          0.32892409, -0.23567124],
        [ 1.12501786,  1.04597301,  1.02666206, ...,  0.69454597,
          1.2105791 , -0.22345278],
        [-2.40317559, -2.94716321, -1.92789435, ..., -4.34043267,
         -2.37973025, -0.88923697]],

       [[ 0.22553994,  0.40935941,  0.58689464, ...,  0.2443013 ,
          0.53541868,  0.92274013],
        [ 0.58809979,  0.26677703,  0.56807097, ...,  0.5591446 ,
          0.92770699,  0.65763429],
        [-1.15078945, -3.89497209, -3.84236794, ..., -2.20261563,
         -1.48111403, -0.30520943]],

       [[ 0.08144092,  0.30741378,  0.40199221, ...,  0.03934218,
          0.37797085,  0.50118016],
        [ 0.15728239,  0.78102987,  0.90211411, ..., -0.02632955,
          0.86598972,  0.56975634],
        [-1.60272921, -6.43621999, -0.57467326, ..., -0.97809619,
         -1.83499698, 